In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import pyjokes
from keras.models import load_model
import json
import random
import tensorflow as tf   
import pandas as pd
tf.get_logger().setLevel('ERROR')
import convert

In [12]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [13]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [14]:
def predict_class(sentence, model , words , classes):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.1
    results = [[i,r] for i,r in enumerate(res) if r>=ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
#     print(return_list)
    return return_list

In [15]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [16]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [17]:
def call_model(sentence , qtype):
    no = qtype
    model = load_model('models/model' + no + '.h5')
    words = pickle.load(open('type' + no + '/words.pkl','rb'))
    classes = pickle.load(open('type' + no + '/classes.pkl','rb'))
    found_intent = predict_class(sentence , model , words , classes)[0]['intent']
#     print("Intent is " , found_intent)
    return found_intent

In [18]:
def main(question):
#     print("Finding the type")
    model = load_model('models/model.h5')
    words = pickle.load(open('main_model/words.pkl','rb'))
    classes = pickle.load(open('main_model/classes.pkl','rb'))
    qtype = predict_class(question , model , words , classes)[0]['intent']
#     print("\nQuestion type is" , qtype , end='\n\n')
    if qtype == 'type1':
        it = call_model(question , '1')
    elif qtype == 'type2':
        it = call_model(question , '2')
    elif qtype == 'type3':
        it = call_model(question , '3')
    return (qtype, it)

In [32]:
main("how are you")

('type1', 'greeting')

In [30]:
def solve(lisp):
    data = {}
    data['question'] = lisp
    data['intent'] = []
    data['types'] = []
    for question in lisp:
        print(question)
        a = main(question)
        data['intent'].append(a[1])
        data['types'].append(a[0])
        if a[1] == 'suggest_movie':
            print('Here is a movie suggestion' , convert.give_suggestion('movies'))
        elif a[1] == 'suggest_song':
            print('Here is a song suggestion' , convert.give_suggestion('songs'))
        elif a[1] == 'suggest_book':
            print('Here is a book suggestion' , convert.give_suggestion('books'))
        elif a[1] == 'tell_joke':
            print("Here is a joke" , pyjokes.get_joke('en'))
    data = pd.DataFrame(data)
    print(data)

In [31]:
lisp = ["campus of iit mandi", "joining programming club ","how are you","why is competetive programming hard", "who are you", "who created you"]
solve(lisp)

campus of iit mandi
joining programming club 
how are you


ValueError: in user code:

    /home/pranshu/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /home/pranshu/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/pranshu/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/pranshu/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/pranshu/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/pranshu/.local/lib/python3.8/site-packages/tensorflow/python/ker

In [180]:
print(convert.give_suggestion('books'))

{'authors': ['Jaideva C. Goswami', ' Andrew K. Chan'], 'image': 'http://libgen.lc//covers/9000/0649d4c0dc548a91adbe1199865dbf9f-d.jpg', 'download': 'http://libgen.lc/ads.php?md5=0649d4c0dc548a91adbe1199865dbf9f', 'publisher': 'Wiley', 'year': '1999', 'pages': '319', 'language': 'English', 'size': '4 MB', 'name': 'Fundamentals of wavelets: theory, algorithms, and applications '}


In [181]:
import wikipedia
print(wikipedia.summary("wikipedia"))

Wikipedia ( (listen) wik-ih-PEE-dee-ə or  (listen) wik-ee-) is a multilingual open-collaborative online encyclopedia created and maintained by a community of volunteer editors using a wiki-based editing system. It is one of the 15 most popular websites as ranked by Alexa, as of January 2021 and The Economist newspaper placed it as the "13th-most-visited place on the web".  Featuring no advertisements, it is hosted by the Wikimedia Foundation, an American non-profit organization funded primarily through donations.
Wikipedia was launched on January 15, 2001, by Jimmy Wales and Larry Sanger. Sanger coined its name as a portmanteau of "wiki" and "encyclopedia". It was initially an English-language encyclopedia, but versions in other languages were quickly developed. With 6.2 million articles, the English Wikipedia is the largest of the more than 300 Wikipedia encyclopedias. Overall, Wikipedia comprises more than 55 million articles, attracting 1.7 billion unique visitors per month.
Wikiped

["Complaining about the lack of smoking shelters, the nicotine addicted Python programmers said there ought to be 'spaces for tabs'.",
 'Ubuntu users are apt to get this joke.',
 'Obfuscated Reality Mappers (ORMs) can be useful database tools.',
 'Asked to explain Unicode during an interview, Geoff went into detail about his final year university project. He was not hired.',
 "Triumphantly, Beth removed Python 2.7 from her server in 2030. 'Finally!' she said with glee, only to see the announcement for Python 4.4.",
 "An SQL query goes into a bar, walks up to two tables and asks, 'Can I join you?'",
 'When your hammer is C++, everything begins to look like a thumb.',
 'If you put a million monkeys at a million keyboards, one of them will eventually write a Java program. The rest of them will write Perl.',
 'To understand recursion you must first understand recursion.',
 "I suggested holding a 'Python Object Oriented Programming Seminar', but the acronym was unpopular.",
 "'Knock, knock.